In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

#from model_prediction import RE
#from model_prediction import Replicator
from pneumococcus_data import data_observation
from Jacobian import jacobian_func

In [2]:
# import Particle Swarm Optimization algorithm
from PSO_algo import Particle
from PSO_algo import PSO

### Description
The number of serotypes reported in Iran is $30$; and $\mu = 0.93$. We need to estimate the matrix $30 \times 30$ $\left(\alpha_{ij}\right)$ of Iran. \
Since $\mu$ of Iran is not large, we estimate parameters such that the data observation (Iran prevalence) is the stable equilibrium of the replicator dynamics. Our process are described as follows: \
i) From $\alpha_{ij}$'s estimated from previous epidemiological sites, in Iran's $\alpha_{ij}$ matrix, we insert $288$ values of common $\alpha_{ij}$'s. \
ii) Then we calculate $\Lambda$ matrix and use PSO algorithm in range $\left[-10, 10\right]$ to estimate $612$ remaining $\alpha_{ij}$'s such that data observation $z^*$ satisfies: $\left\|\Lambda z^* - {z^*}^T\Lambda z^*\right\|_2$ attains its minimum and the equilibrium is stable, i.e. the Jacobian matrix corresponding to the equilibrium $\Lambda z^*$ has all strictly negative real-part eigenvalues. \
In PSO algorithm, we choose the range $\left[-10,10\right]$ for these $612$ $\alpha_{ij}$'s.

In [3]:
# data observation from 5 countries
Iran, Denmark, Brazil, Nepal, Mozambique = data_observation()

# import name of serotypes and their frequencies from Iran
Iran_strain = np.array(Iran.strain)
Iran_cases = np.array(Iran.cases)
strain_number = len(Iran_strain) # = 30

#ratio of single to co-colonization prevalence: mu value of Iran
mu = 0.93

#import Brazil alpha & data
Brazil_alpha = np.loadtxt(open("Brazil_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Brazil_strain = np.array(Brazil.strain)

#import Denmark alpha & data
Denmark_alpha = np.loadtxt(open("Denmark_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Denmark_strain = np.array(Denmark.strain)

#import Mozambique alpha & data
Mozambique_alpha = np.loadtxt(open("Mozambique_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Mozambique_strain = np.array(Mozambique.strain)

In [4]:
# initiate matrix Mozambique_alpha
Iran_alpha = np.ones((strain_number, strain_number))*50

#In Iran alpha matrix, find the positions of alpha_ij already estimated before in Brazil, Denmark, Mozambique and fills with these values
for xI, strain_xI in enumerate(Iran_strain):
    for yI, strain_yI in enumerate(Iran_strain):
        #insert from Brazil
        find_xBrazil = np.where(Brazil_strain == strain_xI)
        index_xBrazil = find_xBrazil[0]

        find_yBrazil = np.where(Brazil_strain == strain_yI)
        index_yBrazil = find_yBrazil[0]

        #insert from Denmark
        find_xDenmark = np.where(Denmark_strain == strain_xI)
        index_xDenmark = find_xDenmark[0]

        find_yDenmark = np.where(Denmark_strain == strain_yI)
        index_yDenmark = find_yDenmark[0]

        #insert from Mozambique
        find_xMozambique = np.where(Mozambique_strain == strain_xI)
        index_xMozambique = find_xMozambique[0]

        find_yMozambique = np.where(Mozambique_strain == strain_yI)
        index_yMozambique = find_yMozambique[0]


        if index_xBrazil.size > 0 and index_yBrazil.size > 0:
            Iran_alpha[xI, yI] = Brazil_alpha[index_xBrazil, index_yBrazil]
        elif index_xDenmark.size > 0 and index_yDenmark.size > 0:
            Iran_alpha[xI, yI] = Denmark_alpha[index_xDenmark, index_yDenmark]
        elif index_xMozambique.size > 0 and index_yMozambique.size > 0:
            Iran_alpha[xI, yI] = Mozambique_alpha[index_xMozambique, index_yMozambique]

#np.savetxt('fix_alpha_Iran.txt', Iran_alpha, delimiter = ',')

In [5]:
#find the length of vector that we need to fit the matrix Iran alpha
length = 0

for i in range(0, strain_number):
    for j in range(0, strain_number):
        if Iran_alpha[i,j] > 49:
            length += 1

print(length)
#so we need to fit the matrix Iran

612


In [6]:
#find the coordinate of missing alpha_ij in Iran alpha_matrix
position = []

for i in range (0, strain_number):
    for j in range (0, strain_number):
        if Iran_alpha[i,j] > 49:
            position.append(np.array([i,j]))

In [7]:
#cost function for PSO
one = np.ones(strain_number) #vector of all ones

def func(vector):
    # insert alpha vector to Iran_alpha
    alpha_matrix = np.loadtxt(open("fix_alpha_Iran.txt","rb"), delimiter = ",", skiprows = 0)
    for i in range (0, length):
        coordinate = position[i]
        alpha_matrix[coordinate[0], coordinate[1]] = vector[i]

    #compute invasion fitness matrix
    lambd = np.zeros((strain_number, strain_number))
    for i in range (0, strain_number):
        for j in range (0, strain_number):
            lambd[i][j] = mu*(alpha_matrix[j][i] - alpha_matrix[i][j]) + alpha_matrix[j][i] - alpha_matrix[j][j]

    # compute the equilibrium of predicted model
    inv = np.linalg.inv(lambd) #inverse matrix of fitness matrix
    z = (1/np.matmul(one.T, np.matmul(inv, one)))*np.matmul(inv, one)
    error = np.linalg.norm(Iran_cases - z)

    #compute Jacobian matrix with the equilibrium
    jac = jacobian_func(lambd, z)
    #compute eigenvalues of Jacobian matrix
    eigenvalues, eigenvectors = LA.eig(jac)
    eig_real = eigenvalues.real
    eig_real = eig_real.tolist()
    #eig_real.append(0)

    stability = max(eig_real)
    return  error, stability #we want minimize error and all negative eigenvalues for the sake of stability

In [8]:
#objective function is base on estimation error and stability (all eigenvalues must have negative real parts)
def func_opt(vector):
    err, stability = func(vector)
    return err + stability*2

In [ ]:
# RUN PSO algo
# initial starting location [x1,x2...]
initial = [0]*length

# input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
bounds = []
for i in range (0, length):
    bounds.append(np.array([-10, 10]))
bounds = np.array(bounds)

# run the algorithm
PSO(func_opt, initial, bounds, num_particles = 4000, maxiter = 2000)

FINAL:
[0.5231095775785812, -0.43408216714133896, 0.01394637032520423, -0.7492866233045767, -1.6093500205717692, -0.3351518004514707, 0.4359223724197183, 1.044783396078349, -0.29361220569343127, -0.8424075400024175, -0.1176553008521714, 1.4004150929743882, -0.5287969026336128, 1.542047279080891, -1.067022329271999, -0.09704560963926458, 0.9479961228785992, 0.08268285819083357, 0.13087311583230227, 0.9739419638591061, -0.6325600254546285, -0.31361773064143184, 0.8719494921612378, 2.410122878911643, 0.2540936196262237, -0.4668886224460882, -0.10027087903736343, -0.7228095017787686, -0.18096642557076487, 0.6548355457201314, 0.48818955093794686, 0.6880887511517116, 1.0080976169639917, 0.9019077802619708, 0.05228995928742893, -1.1532968095170333, 2.0068123512331852, 0.42894601387521025, 0.4416938876300154, -1.386073040819478, 1.0836311677054067, -0.3200314962846544, 0.506265114231204, -1.803346959304497, 0.8416290049244141, -1.2810093680578039, -0.2834492460477336, 0.8646332055701216, -2.79

In [8]:
#the 612-vector predicted
opti_Iran = [0.5231095775785812, -0.43408216714133896, 0.01394637032520423, -0.7492866233045767, -1.6093500205717692, -0.3351518004514707, 0.4359223724197183, 1.044783396078349, -0.29361220569343127, -0.8424075400024175, -0.1176553008521714, 1.4004150929743882, -0.5287969026336128, 1.542047279080891, -1.067022329271999, -0.09704560963926458, 0.9479961228785992, 0.08268285819083357, 0.13087311583230227, 0.9739419638591061, -0.6325600254546285, -0.31361773064143184, 0.8719494921612378, 2.410122878911643, 0.2540936196262237, -0.4668886224460882, -0.10027087903736343, -0.7228095017787686, -0.18096642557076487, 0.6548355457201314, 0.48818955093794686, 0.6880887511517116, 1.0080976169639917, 0.9019077802619708, 0.05228995928742893, -1.1532968095170333, 2.0068123512331852, 0.42894601387521025, 0.4416938876300154, -1.386073040819478, 1.0836311677054067, -0.3200314962846544, 0.506265114231204, -1.803346959304497, 0.8416290049244141, -1.2810093680578039, -0.2834492460477336, 0.8646332055701216, -2.79164050722915, 0.1039168354912168, 0.9802982839974872, -1.7302163915363138, 0.7889145913318248, -1.0677804439953502, -1.493532242944431, -1.1659301930227117, -2.2655374026500086, 0.5667551440774203, -1.5028601965584991, -0.4327702149738491, 0.4179701639003246, 1.1188990553713551, -0.32664978056831884, 2.0041831930688403, 0.47049275566729026, 0.25570967454848387, 0.22746505443596599, 1.4407877543002696, -2.80352157418289, 0.29601063263441246, 0.42977287936901093, -0.6284934997082319, -0.3136846247659612, 0.09569821220412411, -1.1670895918876738, 0.9661701737377294, -1.2232589205891222, -0.6671264918134867, 0.6990349804989476, 1.08083485461482, 1.2097794770770844, -1.0096430665231395, 1.6965615300513845, -0.3176961188815071, -0.8200531419258388, 0.4466304722435618, -1.3668442122609747, 0.378297740801207, 2.007396138120672, -0.28985873815475593, 0.9209802466241238, -0.3320231579656523, 0.6222798383635487, 0.7498250220267795, 1.064451526705074, -1.6074567388506575, -0.7008673401937786, -1.5210018846101458, 0.9272912464097238, -0.3518008347996824, -0.020435902090165128, 1.2890927015354008, 1.3053824805852454, -2.130647085786928, 0.30501520256678427, 1.535229564433606, -0.5654661006289741, -0.21201284177249155, 0.1151190282712505, -0.21369040722427535, -0.6955288762632865, 0.4542366521934881, -2.249184481326912, -0.20536237951088945, 1.374247589332071, 0.15954675379897224, 0.5752691309299013, 0.12481987639318023, 1.1277834357886416, 0.3309583206162655, -1.0730125662681784, 1.7647138804858493, 0.4585035895711986, 0.9418518450329391, -0.7774721837569653, -0.40976710316603854, -0.32660589126542916, 0.8475039922921629, 0.1496251064887544, 0.046132047674438945, -0.430123476321285, -0.46048188280153746, 0.9215051885873586, 0.76795971595257, 0.4693679183643721, -0.029605166452887962, -1.2084624656688823, 0.3063145576202681, 0.262788858262417, 1.0051198384274627, 1.5879460077660268, -0.8078903570297665, 0.9591425420885116, -0.2515715711811833, -1.0392556915039222, 1.3857169980448159, -0.6023138650363676, 1.9226744335381767, 3.3398621866754334, 1.018224828167945, 0.22062655339742065, -0.02214214688323209, 0.9408751388189525, 3.813303743118213, -0.6862824053105228, -0.2463709945307937, -0.7852767254684668, 0.46602585294302773, -0.06606782614816148, 2.8201985827615625, 0.2838165896032813, -0.3827123767212496, 0.3474088797704119, -0.17528356633497602, -0.7896129715916462, 0.1755988460471577, 0.40939849415249324, 1.3123543256773504, 1.619340351879322, 1.5791353276691384, 0.1146487312183189, 3.050165318847352, 0.410282904753878, -0.1468113997822989, -1.755996733295052, 0.02943435793746705, -1.1715764449786126, 2.6150290839074604, -0.45152504942257404, 0.13864802667656204, 0.4151755096091424, -0.9841471251734972, -0.20012752852490834, -0.0483768250862337, -1.021757191954482, 0.3248582928364484, -0.49841914844961827, 0.7082695904816455, -1.8438667211781343, -0.21857861391478167, 0.4630448240758948, 0.28570050190352775, -0.035809404580438194, -0.25040408582896173, 1.0549080998288525, -1.2412151369337459, 1.5474288459377576, 1.013241595823053, 0.7388860632338622, -1.1048323750425089, 2.9754176277547373, -1.6170448577775074, -0.17299147769116355, 0.9630508137436757, -0.7751574335654918, -1.1259472695076183, -1.3349881748441381, 0.192150068228018, 0.3733447697720619, 0.9799633987977704, -0.4186933887046456, -0.5336582227551938, 0.8868296893367126, 3.43146445583279, 0.0884271205007687, -4.005475643654617, 1.131568777866095, -0.1453818952213033, -1.8058451161755638, -1.1856809709020695, 0.4489835702394395, 0.4037674413665811, -5.032984049895617, 2.1732731575642075, 0.6131801612258363, -0.3367434330507018, -0.8742768778859258, -0.8182707784457381, -1.1042793373145892, 0.38589237776672836, -0.04627490296900067, -1.508071409204611, -1.3702795074948313, -1.3245604384648428, -0.36797869566805924, 0.9406786354070988, 3.1833986733903834, -0.5072701071645146, 0.9753319262701076, -0.5794492684030313, 1.0805695015803927, 0.22860923997201038, 0.661593126786324, 0.1836868131309094, 1.1690145075295948, 1.2441245962314529, -0.31649400885070816, -1.394152625924564, -0.5489399026044467, -0.22279169676455227, -0.76514008453436, 2.3824521778899213, -1.3350615996492086, -2.283691339355135, -0.4846060032843376, 0.7332315890645245, -0.07046607632270012, -0.22996026768999753, -1.487550215015033, -1.2372561620760456, -2.072138963465321, -0.6470542252188092, -0.20730411106215924, -1.0961340890262004, -0.23464827818265022, -3.1878559885549658, -0.052682246028769045, 1.2242052247064303, 0.9457400815821537, -0.039798785250501484, 1.5156104983811294, -0.19555988808237637, 1.774634948736296, -0.10605445014699587, 0.07228222961345565, 0.18629054279682883, 0.07774303000058523, -0.9939765109728943, -0.3482626065336622, 1.0506805965826382, -0.5003113284672073, -0.04137675203983194, 0.21497845641747432, 1.2313779600604016, 1.793423391198926, -0.5437046673678521, 0.9486636470676162, -0.21068171084882098, -0.8379776229069312, 2.661783718446965, -0.6965216052699353, -0.4025761192157531, -0.039635755135128715, -0.37909267284222875, -0.48122754818511987, -0.31363863976724715, 2.117674919859676, -0.40268833860481684, -1.8961357456692878, 0.030499567785265438, -1.0736921249791938, 0.09913196024801496, -0.6911338941130095, 0.10857197097593506, 1.6688219323705202, -0.9750584578779243, 1.14491089664391, 0.7545736218266592, -0.6397291197640055, 0.08747018646015602, 1.0737115945534939, 2.6049795071809476, 1.1149744575937812, -2.1804880387544427, -0.27129154179657305, 1.444591108867493, 2.4019635810229887, 1.890087095170694, -0.3735720428482107, -1.0381118192182144, 0.44684910243100484, 2.2695318750936657, -1.0145650456659705, -0.06225868205754118, 1.6384007614488634, 0.3680933491986114, 1.0979945675612466, -0.24725143543706973, -0.3517559973833048, -1.9006474993515952, 2.110541160973262, -0.455797862065913, -0.5069978333023681, -2.2914419959539054, -0.27221524057576585, -0.2091912321573966, 1.6033289728105524, 0.49085141288562095, -1.8297984779394374, -2.8030397651197023, -1.4358317220955283, -0.08642181029724842, -0.8305995192682892, -0.30976580422384553, 2.03749871505598, -0.21485333924744002, 0.13990475730589955, -0.05880098753173166, -0.2589510155712138, 1.1621068119094922, -0.5732487368078832, 0.22895559975425164, 1.137814264650339, 0.6057399285598771, -1.1541539505386906, 0.4712430067413706, 0.9911602864370668, -3.9790793155449777, 0.36283838148746766, 0.3754028903834523, 1.1198936302170013, 0.3311462591771381, -0.8528820277450507, -0.8726563799700133, 1.904167372069471, 1.551455591688685, -1.7989871477056019, -0.34839386117727317, -1.7068267907758712, -0.6451364962625467, 0.3439946354512677, 1.77966084663634, 0.16725570515760596, 0.8508529523935701, -0.7230790439601466, 1.0056686380440303, -4.1287891523607305, 0.7625084277835791, -0.15308668528185293, 1.300055440041774, -1.0628181619773818, -1.3087117874233833, 1.6413189724164483, 0.22892675669629733, -1.7675917916780222, -1.8270333778664045, -0.2834630114692563, 1.6271280912203394, -0.5690012392862066, -1.1537552129230493, -0.1531427937919238, 1.094645172077101, -0.28089657432190623, 1.4388142770796022, 2.3108017769635096, 2.9871716030823814, 0.07159187347677456, -0.7985939262535835, -1.1076318796975748, -0.30007772783375875, 5.454273690223248, -1.8230163633231238, -2.592052005440254, -1.1443988679372668, 0.5401263635963813, 0.4180138061847744, 2.433324970526977, -2.7712397785757887, -2.905189129485676, 2.0627428444559786, -1.1232039748643154, 0.6217584120635983, 2.961834998937287, 0.5712077957366383, 0.4463672864621386, -0.6236444059119665, 0.04082358798845979, 2.4666158096626267, 0.39130946183536747, 0.0684282442107354, -0.44326269498829757, -0.7438633755904779, -1.9607826503227233, -0.5896252369500526, 0.1675062127950044, -0.6372783484529715, -0.256418232929131, -4.859213039437121, 0.8180676851179314, -0.4062245032875331, -1.8428675215161958, 2.1891599971753575, 1.4659460849873727, -0.3959246450863284, 0.24012971506074954, 0.9707261005998883, -0.3060592041970804, 0.17394227809942256, -0.14091131769259715, -1.5804651571763828, -2.069755880518852, -0.4133763620224859, 1.2236604031472553, -0.10943258488123098, 1.232994244060147, 0.15949909097796533, -1.525720268594196, -0.47154775455942866, 0.16034662283477188, -0.409093696911472, -2.3526975765254194, -0.9831289719780871, 1.8787257113858948, 1.6875358992536673, -1.2003472242109978, 0.1375137224699492, -0.17996481080138213, -0.6200895986896832, 0.04145573477461562, 1.249171572225133, -1.2722026512478513, -0.3354039021728429, 0.24952796861183166, 0.9340232332637748, -0.5528424472624036, -0.33162803388267126, 0.3703854933766484, -3.8366548866761803, 0.2810004121379746, -2.2858537985687937, -2.330595060927548, -1.3315963482349442, -0.6440763205380389, -0.8860889086191152, 1.4824819774877738, 0.8336631768392372, -1.505617401615048, -2.590084376711004, -1.189803800805099, -1.492492624074934, -1.602677261374477, -1.846879242386511, 1.4844312529596224, -0.5272623126634696, -0.3320472813673988, -0.7461427810360727, 0.5878635324132305, -3.017526666198101, 1.2874002687135973, -1.4948353459370103, 1.2319126422278777, 0.19132128942663154, 1.9291762881816905, -1.6980126333295775, 0.5983257411352729, -0.8205295901337814, -1.4999543854079156, -0.9627610197626922, 0.5756135681507272, -0.4627962775850984, 0.46575127868474164, -0.2502981301287277, 0.6203800856133229, 0.6763532798317575, 0.5750680097427663, -1.0538718247073386, 0.36094313521694577, 1.2676116254521934, -2.7761411608745146, -0.7922332722120797, 1.7928507227245365, -0.0947585352158086, 0.15819216935909175, -3.4599518424645788, -0.5746876963326322, 0.22374744169025576, 0.3212667196559001, -0.5189494818137502, 1.0823901301630676, -1.8687180474703502, 0.17057876136807934, 0.270802680424607, -0.6136469782869333, -0.3967797881947473, 0.1704057714158819, 1.532304016648206, 0.19213214143897897, -1.5152916538427157, 0.08950959050680452, 1.9080371167407457, 0.7610306778979743, 0.07135108141933243, 0.3570997875292319, -0.0623178483622524, 1.1894842563852952, -0.49359190710658934, 2.0387301702966596, -0.27823743824338626, -0.7735701880960739, 3.6306870144639767, -0.15153536282492194, 2.0723614171531115, 0.470725753510628, 0.4700534616988876, 1.1812719435520813, 4.685511126896025, 1.6176556677203409, 2.0852940452458726, 0.8007709676663513, 0.8992929048054457, -1.3600030235830165, 0.8814405702146719, 1.4125792286832999, 0.810147049341611, -0.9008414971321752, -1.7402197711446992, 0.6385861413049816, 0.6234432011848556, 0.2616839026814506, 1.500134571785627, -0.1883263065255184, -0.5722606940498751, 2.251735789096888, 0.5705002490579225, -1.091887862631333, -0.4066359861703859, 0.43262028322868074, 0.05763354320140599, -0.1882825308492729, 0.19583603075940184, 0.11698857437129487, 0.32154962304729556, -0.06295223800103437, 0.1572276905611318, -0.7321801256672319, -0.31599138488578654, -2.6032396521835675, 2.4920109753506368, 0.5255133037427566, 1.1249966652436687, -1.0462786164571956, 0.20531820083312932, 0.3155160535573975, 0.5616389488354823, -0.3669985876785309, 1.1542559495446598, -1.5051094752961964, 2.8200292832550757, 0.287108921084533, -0.590412005925411, -0.20196540772009924, -0.6962116445751952, -0.24277944467674162, 0.37644865817036177, -4.077212566773946, -0.43232024195671986, -0.3196788342421598, -0.35272900509297744, -2.6085263044655154, -0.5784560397465289, 1.106360097887601, 0.20267608612926377, 1.0634475405008725, -0.7592784800111374, 0.7651556052669296, 0.5831469564604596, -3.7004957812350083, 0.38662947056652625, 0.2148386307471321, 2.095652571173177, -0.03773464454781127, 1.5811678937078935]
#load the Iran alpha matrix which already has the common alpha_ij inserted from Brazil, Denmark and Mozambique alphas
Iran_alpha = np.loadtxt(open("fix_alpha_Iran.txt","rb"), delimiter=",", skiprows = 0)

#insert 612 alpha_ij predicted in this step
for i in range (0, length):
        coordinate = position[i]
        Iran_alpha[coordinate[0], coordinate[1]] = opti_Iran[i]

#save Iran alpha matrix
np.savetxt('Iran_alpha.txt', Iran_alpha, delimiter = ',')

In [8]:
# test
# import Iran alpha matrix
Iran_alpha = np.loadtxt(open("Iran_alpha.txt","rb"), delimiter = ",", skiprows = 0)

#Compute the fitness matrix
lambd = np.zeros((strain_number, strain_number))
for i in range (0, strain_number):
    for j in range (0, strain_number):
        lambd[i][j] = mu*(Iran_alpha[j][i] - Iran_alpha[i][j]) + Iran_alpha[j][i] - Iran_alpha[j][j]

# compute the equilibrium of predicted model
one = np.ones(strain_number) #vector of all ones
inv = np.linalg.inv(lambd) #inverse matrix of fitness matrix
z = (1/np.matmul(one.T, np.matmul(inv, one)))*np.matmul(inv, one)

#compute error between observation data and equilibrium of predicted model
error = Iran_cases - z
error = np.linalg.norm(error)
print(error)

0.014594124413556772


In [12]:
#compute Jacobian matrix with Iran's frequencies
jac = jacobian_func(lambd, z)

#compute eigenvalues of Jacobian matrix
eigenvalues, eigenvectors = LA.eig(jac)
real_eig = eigenvalues.real
eig = real_eig.tolist()
#find the maximum of real parts and print it
max_eig = max(eig)
print(max_eig)

-0.0016438881308544068


In [13]:
#print all real parts of eigenvalues
print(eig)

[-0.029394646601449383, -0.029394646601449383, -0.0016438886257263599, -0.0016438886257263599, -0.006300349983835063, -0.006300349983835063, -0.3646278853586485, -0.002686062717815318, -0.002686062717815318, -0.004281124893620108, -0.004281124893620108, -0.002096711202638327, -0.002096711202638327, -0.0022968722905445395, -0.0022968722905445395, -0.044577655981165094, -0.044577655981165094, -0.020894045049022997, -0.020894045049022997, -0.05970490632196726, -0.05970490632196726, -0.0016438881309249044, -0.0016438881309249044, -0.0016438881308544068, -0.0016438881308544068, -0.0016438881310328207, -0.0016438881310328207, -0.0016438881308887263, -0.0016438881308887263, -0.0037242529757950038]


In [14]:
#test if there is any real parts non-negative
positive_eig = [i for i in eig if i >= 0]
print(positive_eig, len(positive_eig))

[] 0
